In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_ollama  import OllamaEmbeddings
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.documents import Document
import re
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain.document_loaders import TextLoader

In [49]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)

In [50]:
# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

In [51]:
# Text Split (문서 분할)
def split_cafe_menu(document):
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        price_match = re.search(r"• 가격:\s*₩([\d,]+)", item)
        price = int(price_match.group(1).replace(",", "")) if price_match else 0
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name,
                "price": price, # 가격 메타데이터 추가
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents

menu_documents=[]
for doc in documents:
    menu_documents += split_cafe_menu(doc)
    
# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...

메뉴 번호: 2
메뉴 이름: 카페라떼
내용:
2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다...


In [52]:
embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
cafe_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장
cafe_db.save_local("./db/cafe_db")

In [53]:
cafe_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model,
    allow_dangerous_deserialization=True
)

print(cafe_db.similarity_search("아메리카노", k=3))

[Document(id='f499c735-0eb8-4e56-ae06-38d1f0c1f1ff', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 1, 'menu_name': '아메리카노', 'price': 4500}, page_content='1. 아메리카노\n   • 가격: ₩4,500\n   • 주요 원료: 에스프레소, 뜨거운 물\n   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.'), Document(id='20339a8c-94d4-46a6-b2ec-38a778f9a416', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 9, 'menu_name': '아이스 아메리카노', 'price': 4500}, page_content='9. 아이스 아메리카노\n   • 가격: ₩4,500\n   • 주요 원료: 에스프레소, 차가운 물, 얼음\n   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.'), Document(id='a9ff3ce5-179f-4064-9aa2-b61fcb5a6624', metadata={'source': '../data/cafe_menu_data.txt', 'menu_number': 3, 'menu_name': '카푸치노', 'price': 5000}, page_content='3. 카푸치노\n   • 가격: ₩5,000\n   • 주요 원료: 에스프레소, 스팀 밀크, 우유 거품\n   • 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 

In [54]:
# Tool 정의 
@tool
def search_cafe_menu(query: str) -> str:
    """
    카페 메뉴 정보(이름, 설명, 가격, 카테고리 등)를 검색합니다.
    사용자가 특정 음료, 디저트 또는 메뉴 특징에 대해 질문하면 이 도구를 사용하세요.
    예: '아메리카노 가격', '달콤한 디저트 추천', '라떼 종류'
    """
    print(f"\n--- Calling search_cafe_menu tool with query: '{query}' ---")
    docs = cafe_db.similarity_search(query, k=3) # Retrieve top 3 relevant documents
    if not docs:
        return "죄송합니다, 관련된 메뉴 정보를 찾을 수 없습니다."
    
    result_str = "검색된 메뉴 정보:\n"
    for doc in docs:
        result_str += f"\n---\n{doc.page_content}\n---\n"
    print(f"Tool Result: {result_str}")
    return result_str

In [55]:
# 도구 목록
tools = [search_cafe_menu]

# 모델에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=tools)
print(llm_with_tools)

bound=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001BF6218F1A0>, default_metadata=(), model_kwargs={}) kwargs={'tools': [{'type': 'function', 'function': {'name': 'search_cafe_menu', 'description': "카페 메뉴 정보(이름, 설명, 가격, 카테고리 등)를 검색합니다.\n사용자가 특정 음료, 디저트 또는 메뉴 특징에 대해 질문하면 이 도구를 사용하세요.\n예: '아메리카노 가격', '달콤한 디저트 추천', '라떼 종류'", 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]} config={} config_factories=[]


In [56]:
class AgentState(MessagesState):
    pass

In [57]:
def agent_node(state: AgentState):
    """
    Agent 노드: LLM을 호출하여 응답을 생성하거나 도구를 호출합니다.
    """
    print("\n--- Agent Node Running ---")
    current_messages = state['messages']
    
    if len(current_messages) == 1 and isinstance(current_messages[0], HumanMessage):
        system_prompt = HumanMessage(
            content=(
                "당신은 친절한 카페 직원입니다. "
                "고객의 질문에 답변하고, 메뉴 정보가 필요하면 'search_cafe_menu' 도구를 사용하세요. "
                "도구 사용 후에는 도구 결과를 바탕으로 최종 답변을 생성해주세요."
            )
        )
        pass
    
    response_message = llm_with_tools.invoke(current_messages)
    
    print(f"결과: {response_message.pretty_repr()}")
    
    return {"messages": [response_message]}

In [58]:
from langgraph.prebuilt import ToolNode, tools_condition

tool_node = ToolNode(tools=[search_cafe_menu])
builder = StateGraph(AgentState)

builder.add_node("agent", agent_node)
builder.add_node("tools", tool_node)
builder.set_entry_point("agent")

builder.add_conditional_edges(
    "agent", 
    tools_condition,
    {
        "tools": "tools",
        END: END
    }
)

builder.add_edge("tools", "agent")

graph = builder.compile()

In [59]:
test_queries = [
    "아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.",
    "라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?",
    "디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요."
]

In [60]:
# 각 질문에 대해 그래프 실행
for query in test_queries:
    print(f"\n질문: {query}")
    inputs = {"messages": [HumanMessage(content=query)]}
    messages = graph.invoke(inputs)
    for m in messages['messages']:
        print(m)


질문: 아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.

--- Agent Node Running ---
결과: ================================== Ai Message ==================================

아메리카노와 아이스 아메리카노는 둘 다 에스프레소에 물을 넣어 만드는 음료입니다. 가장 큰 차이점은 아메리카노는 따뜻하게 제공되고, 아이스 아메리카노는 차갑게 제공된다는 점입니다.

가격은 카페마다 다를 수 있습니다. 어떤 카페의 아메리카노와 아이스 아메리카노 가격이 궁금하신가요? 특정 카페를 알려주시면 메뉴 정보를 검색해 알려드릴 수 있습니다.
content='아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.' additional_kwargs={} response_metadata={} id='8065eb51-3364-491a-ab0b-8f910dd80c69'
content='아메리카노와 아이스 아메리카노는 둘 다 에스프레소에 물을 넣어 만드는 음료입니다. 가장 큰 차이점은 아메리카노는 따뜻하게 제공되고, 아이스 아메리카노는 차갑게 제공된다는 점입니다.\n\n가격은 카페마다 다를 수 있습니다. 어떤 카페의 아메리카노와 아이스 아메리카노 가격이 궁금하신가요? 특정 카페를 알려주시면 메뉴 정보를 검색해 알려드릴 수 있습니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--4176efa1-df87-4e62-83b2-195c2021a149-0' usage_metadata={'input_tokens': 116, 'output_tokens': 137, 'total_tokens': 253, 'inpu